In [ ]:
# Script para descargar información de bancos de la Superfinanciera y mostrar alertas
# TODO: incluir dataframe de pandas, analizar principales cifras

import bs4, requests, zipfile, os, re

# Parte para descargar archivos. Revisa cuáles hay en la página y descarga los que no hay en la carpeta

direccion = 'https://www.superfinanciera.gov.co/publicacion/10084375'
direccion_base = 'http://www.superfinanciera.gov.co'
directorio_archivos = 'C:\\Users\\Juan\\Documents\\sfc\\base'

pagina = requests.get(direccion) # descarga el HTML

pagina.raise_for_status() # check errores

soup = bs4.BeautifulSoup(pagina.text, 'html.parser') # parse HTML

links = [x.attrs['href'] for x in soup.select('.pub li a')] # extrae links de acuerdo al DOM de la página y crea array con links a archivo (.zip), sin dirección base

archivos_actuales = [x[:x.rfind('.')] for x in os.listdir(directorio_archivos)] # lista de archivos en directorio_archivos, sin extensión

links_dict = {re.search('00[0-9]*n',x)[0]:x for x in links} # diccionario de: (nombre de archivo sin extensión : link para descargar)

numero_links = len(links)

links_descargar = [x for x in links_dict.keys() if x not in archivos_actuales]

#fechas_array = [arreglar_fecha(x[-13:-5]) for x in links] # array de fechas en formato ddmmaaaa

os.chdir(directorio_archivos)

lista_archivos = []

if len(links_descargar) > 0:
    print('Descargando', str(numero_links - len(archivos_actuales)), 'archivos' if len(links_descargar) > 1 else 'archivo', end = '')
    for x in links_descargar:
        # baja los archivos a directorio_archivos, extrae el zip,
        # anota el nombre del .xls en lista_archivos y borra el zip
        temp_bytes = requests.get(direccion_base + links_dict[x])
        temp_bytes.raise_for_status()
        temp_zip = open(directorio_archivos + x[-15:], 'wb')
        temp_zip.write(temp_bytes.content)
        z = zipfile.ZipFile(temp_zip.name, 'r')
        z.extractall(path = directorio_archivos)
        temp_zip.close()
        lista_archivos.append(z.namelist()[0])
        z.close()
        os.remove(temp_zip.name)
        print('.', end = '')
        if x == links_descargar[len(links_descargar) - 1]: print('ok')
else:
    print('Archivos al día')

#del archivos_actuales, direccion, direccion_base, links, links_descargar, links_dict, numero_links

In [44]:
import xlrd, datetime, os, json, copy

def extension(nombre_archivo): # devuelve la extensión de un nombre de archivo
    return nombre_archivo[nombre_archivo.find('.'):]

def arreglar_fecha(nombre_archivo): # arregla las fechas de 'ddmmaaaa.xls' a 'aaaammdd'
    x1 = nombre_archivo[2:nombre_archivo.find('.')-1]
    x2 = x1[-4:] + x1[2:4] + x1[:2]
    return x2

nombres_cambiados = {}
def buscar_nombres(nuevo):
    global nombres_cambiados
    global entidades
    esta = False
    anterior = None
    borrar_anterior = None
    for n in entidades:
        numero_entidad = n[:n.find('-')]
        if numero_entidad == nuevo[:nuevo.find('-')]:
            esta = True
            anterior = copy.copy(n)
            borrar_anterior = n
            if numero_entidad not in nombres_cambiados.keys():
                nombres_cambiados[numero_entidad] = [anterior]
            if nuevo not in nombres_cambiados[numero_entidad]:
                nombres_cambiados[numero_entidad].append(nuevo)
    if esta:
        print('del:', borrar_anterior, '\n', entidades)
        del borrar_anterior
    else: entidades.append(nuevo)
    return [esta, anterior]

def fecha_datetime(texto): # recibe 00ddmmaaaa.xls y devuelve objeto datetime
    return datetime.date(year = int(texto[-4:]), month = int(texto[2:4]), day = int(texto[:2]))

directorio_archivos = 'C:\\Users\\Juan\\Documents\\sfc\\base\\'
directorio_txt = 'C:\\Users\\Juan\\Documents\\sfc\\'

lista_archivos = [x for x in os.listdir(directorio_archivos) if extension(x)=='.xls']
numero_archivos = len(lista_archivos)

if numero_archivos:
    print('Leyendo', numero_archivos, 'archivos' if numero_archivos > 1 else 'archivo')

    lista_archivos.sort(key = arreglar_fecha)

c = 0

cartera = ['140400','140800','141000','141200','141400']
entidades = []
activos = {}
cartera = {}

for nombre_archivo in lista_archivos[10:]:
    excel = xlrd.open_workbook(directorio_archivos + nombre_archivo)
    hoja = excel.sheets()[0]
    filadatos = 0
    fecha_str = arreglar_fecha(nombre_archivo)
    for fila in range(15): # busca en las primeras 15 filas "ACTIVO" para saber en cual comienzan los datos
        if hoja.cell(fila, 1).value == 'ACTIVO':
            filadatos = fila
    for columna in range(2, hoja.ncols):
        nombre_entidad = hoja.cell(filadatos-1, columna).value
        if nombre_entidad != 'TOTAL':
            if nombre_entidad not in entidades:
                x = buscar_nombres(nombre_entidad)
                if not x[0]:
                    activos[nombre_entidad] = {}
                    cartera[nombre_entidad] = {}
                else:
                    print("nombre_entidad:", nombre_entidad, "x[1]:", x[1])
                    activos[nombre_entidad] = activos.pop(x[1])
                    cartera[nombre_entidad] = cartera.pop(x[1])
            cartera[nombre_entidad][fecha_str] = 0
            for fila in range(filadatos, hoja.nrows):
                cuenta_puc = str(int(hoja.cell(fila, 0).value))
                nombre_cuenta = hoja.cell(fila, 1).value
                if cuenta_puc == '100000':
                    activos[nombre_entidad][fecha_str] = hoja.cell(fila, columna).value * 1000
                if cuenta_puc in cartera:
                    cartera[nombre_entidad][fecha_str] += hoja.cell(fila, columna).value * 1000
    c += 1
    print(c, end = ' ')
print('')

print("Nombres cambiados:\n", repr(nombres_cambiados))
json_activos = open(directorio_txt + 'activos.json', 'w', encoding = 'utf8')
json.dump(activos, json_activos, ensure_ascii=False, indent=2)
json_cartera = open(directorio_txt + 'cartera.json', 'w', encoding = 'utf8')
json.dump(cartera, json_cartera, ensure_ascii=False, indent=2)
json_activos.close()
json_cartera.close()

Leyendo 41 archivos
1 del: 2- BANCO POPULAR S.A. 
 ['1- BANCO DE BOGOTA', '2- BANCO POPULAR S.A.', '6- BANCO CORPBANCA', '7- BANCOLOMBIA S.A.', '9- CITIBANK', '12- BANCO GNB SUDAMERIS S.A.', '13- BBVA COLOMBIA', '23- Banco  de  Occidente S.A.', '30- BANCO CAJA SOCIAL BCSC', '39- DAVIVIENDA S.A.', '42- Banco Colpatria', '43- BANCO AGRARIO DE COLOMBIA S.A.', '49- AV VILLAS', '51- BANCO PROCREDIT COLOMBIA', '52- BANCAMIA', '53- BANCO WWB S.A.', '54- BANCO COOMEVA S.A BANCOOMEVA', '55- BANCO FINANDINA S.A.', '56- BANCO FALABELLA', '57- BANCO PICHINCHA S.A.', '58- BANCO COOPERATIVO COOPCENTRAL', '59- BANCO SANTANDER DE NEGOCIOS COLOMBI', '60- BANCO MUNDO MUJER S.A', '61- BANCO MULTIBANK', '62- BANCO COMPARTIR S.A.']
nombre_entidad: 2- BANCO POPULAR x[1]: 2- BANCO POPULAR S.A.
del: 7- BANCOLOMBIA S.A. 
 ['1- BANCO DE BOGOTA', '2- BANCO POPULAR S.A.', '6- BANCO CORPBANCA', '7- BANCOLOMBIA S.A.', '9- CITIBANK', '12- BANCO GNB SUDAMERIS S.A.', '13- BBVA COLOMBIA', '23- Banco  de  Occidente S.A.

2 del: 2- BANCO POPULAR S.A. 
 ['1- BANCO DE BOGOTA', '2- BANCO POPULAR S.A.', '6- BANCO CORPBANCA', '7- BANCOLOMBIA S.A.', '9- CITIBANK', '12- BANCO GNB SUDAMERIS S.A.', '13- BBVA COLOMBIA', '23- Banco  de  Occidente S.A.', '30- BANCO CAJA SOCIAL BCSC', '39- DAVIVIENDA S.A.', '42- Banco Colpatria', '43- BANCO AGRARIO DE COLOMBIA S.A.', '49- AV VILLAS', '51- BANCO PROCREDIT COLOMBIA', '52- BANCAMIA', '53- BANCO WWB S.A.', '54- BANCO COOMEVA S.A BANCOOMEVA', '55- BANCO FINANDINA S.A.', '56- BANCO FALABELLA', '57- BANCO PICHINCHA S.A.', '58- BANCO COOPERATIVO COOPCENTRAL', '59- BANCO SANTANDER DE NEGOCIOS COLOMBI', '60- BANCO MUNDO MUJER S.A', '61- BANCO MULTIBANK', '62- BANCO COMPARTIR S.A.']
nombre_entidad: 2- BANCO POPULAR x[1]: 2- BANCO POPULAR S.A.


KeyError: '2- BANCO POPULAR S.A.'

In [29]:
#Archivo HTML de cuentas PUC (100000  ACTIVOS)
import xlrd
excel = xlrd.open_workbook("C:\\Users\\Juan\\Documents\\sfc\\base\\0001052018n.xls")
cuentas = open("C:\\Users\\Juan\\Documents\\sfc\\cuentas.html", "w", encoding = "utf8")
print("""<!DOCTYPE html>
<html>
  <head>
  <meta charset="utf-8">
  <title>Cuentas PUC</title>
    <style>
      .puc {margin-right: 10px;}
      .nivel1 {background-color: black; color: white;}
      .nivel2 {background-color: #aaa;}
      span {font-family: monospace; font-size: 1.2em;}
      body {overflow: scroll;}
    </style>
  </head>
  <body>
  """, file=cuentas, end="")

filadatos = 0
hoja = excel.sheets()[0]
for fila in range(15): # busca en las primeras 15 filas "ACTIVO" para saber en cual comienzan los datos
    if hoja.cell(fila, 1).value == 'ACTIVO':
        filadatos = fila
for fila in range(filadatos, hoja.nrows):
    cuenta_puc = str(int(hoja.cell(fila, 0).value))
    nombre_cuenta = hoja.cell(fila, 1).value
    n = ''
    if cuenta_puc[-4:] == '0000':
        n = ' nivel1'
    elif cuenta_puc[-2:] == '00':
        n = ' nivel2'
    else:
        n = ''
    print("<span class=\"puc" + n + "\">" + cuenta_puc + "</span><span class=\"" + n + "\">" + nombre_cuenta + "</span>", file=cuentas, end="<br>")

print("""  </body>
</html>""", file=cuentas, end="")
cuentas.close()

In [40]:
print(nombres_cambiados)

{'2': ['2- BANCO POPULAR S.A.', '2- BANCO POPULAR', '2- BANCO POPULAR'], '7': ['7- BANCOLOMBIA S.A.', '7- BANCOLOMBIA'], '12': ['12- BANCO GNB SUDAMERIS S.A.', '12- BANCO GNB SUDAMERIS'], '23': ['23- Banco  de  Occidente S.A.', '23- BANCO DE OCCIDENTE'], '30': ['30- BANCO CAJA SOCIAL BCSC', '30- BCSC S.A.'], '39': ['39- DAVIVIENDA S.A.', '39- DAVIVIENDA'], '42': ['42- Banco Colpatria', '42- COLPATRIA RED MULTIBANCA'], '43': ['43- BANCO AGRARIO DE COLOMBIA S.A.', '43- BANAGRARIO'], '51': ['51- BANCO PROCREDIT COLOMBIA', '51- PROCREDIT'], '52': ['52- BANCAMIA', '52- BANCAMIA S.A.'], '53': ['53- BANCO WWB S.A.', '53- WWB S.A.'], '54': ['54- BANCO COOMEVA S.A BANCOOMEVA', '54- BANCOOMEVA'], '55': ['55- BANCO FINANDINA S.A.', '55- FINANDINA'], '56': ['56- BANCO FALABELLA', '56- BANCO FALABELLA S.A.'], '58': ['58- BANCO COOPERATIVO COOPCENTRAL', '58- COOPCENTRAL'], '59': ['59- BANCO SANTANDER DE NEGOCIOS COLOMBI', '59- BANCO SANTANDER'], '60': ['60- BANCO MUNDO MUJER S.A', '60- MUNDO MUJER']